# Numerical Differentiation

In [ ]:
import numpy as np

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
#import pystatsml.plot_utils

# Plot parameters
plt.style.use('seaborn-v0_8-whitegrid')
fig_w, fig_h = plt.rcParams.get('figure.figsize')
plt.rcParams['figure.figsize'] = (fig_w, fig_h * .5)
%matplotlib inline

Sources:

- [Patrick Walls course](https://patrickwalls.github.io/mathematicalpython/differentiation/differentiation/) of Dept of Mathematics, University of British Columbia.
- [Wikipedia](https://en.wikipedia.org/wiki/Numerical_differentiation)

The derivative of a function  at is the limit

$$
f'(x) = \lim_{h \to 0} \frac{f(x+h) - f(x)}{h}
$$

For a fixed step size $h$, the previous formula provides the slope of the function using the forward difference approximation of the derivative. Equivalently, the slope could be estimated using backward approximation with positions $x - h$ and $x$.

The most efficient numerical derivative use the central difference formula with step size is the average of the forward and backward approximation (known as symmetric difference quotient):

$$
f'(a) \approx \frac{1}{2} \left( \frac{f(a + h) - f(a)}{h} + \frac{f(a) - f(a - h)}{h} \right) = \frac{f(a + h) - f(a - h)}{2h}
$$

Chose the step size depends of two issues


1. Numerical precision: if $h$ is chosen too small, the subtraction will yield a large rounding error due to cancellation will produce a value of zero. For basic central differences, the optimal (Sauer, Timothy (2012). Numerical Analysis. Pearson. p.248.) step is the cube-root of machine epsilon ($2.2*10^{-16}$ for double precision), i.e.: $h\approx 10^{-5}$:


In [ ]:
eps = np.finfo(np.float64).eps
print("Machine epsilon: {:e}, Min step size: {:e}".format(eps, np.cbrt(eps)))


2. The error of the central difference approximation is upper bounded by a function in $\mathcal{O}(h^2)$. I.e., large step size $h=10^{-2}$  leads to large error of $10^{-4}$. Small step size e.g., $h=10^{-4}$ provide accurate slope estimation in $10^{-8}$.

Those two points argue for a step size $h \in [10^{-3}, 10^{-6}]$

Numpy [gradient](https://numpy.org/doc/stable/reference/generated/numpy.gradient.html) function for unmerical

In [ ]:
range_ = [-3, 3]
dx = 1e-3
n = int((range_[1] - range_[0]) / dx)
x = np.linspace(range_[0], range_[1], n)
f = lambda x: 3*np.exp(x) / (x**2 + x + 1)

y = f(x)
dydx = np.gradient(y, dx)

plt.plot(x, y)
plt.plot(x[1:-1], dydx[1:-1])
plt.show()

Example: Numerical differenciation of the function
 
$$
f(x) = \frac{7x^3-5x+1}{2x^4+x^2+1} \ , \ x \in [-5,5]
$$

In [ ]:
range_ = [-5, 5]
dx = 1e-3
n = int((range_[1] - range_[0]) / dx) 
x = np.linspace(range_[0], range_[1], n)
f = lambda x:  (7 * x ** 3  - 5 * x + 1) / (2 * x ** 4 + x ** 2 + 1)

y = f(x)
dydx = np.gradient(y, dx)

Use Symbolic Differentiation with sympy to compute true derivative $f'$:

In [ ]:
import sympy as sp
from sympy import lambdify
x_s  = sp.symbols('x', real=True) # defining the variables

f_sym = (7 * x_s ** 3  - 5 * x_s + 1) / (2 * x_s ** 4 + x_s ** 2 + 1)
df_sym =  sp.simplify(sp.diff(f_sym))
print("f =", f_sym)
print("f'=", df_sym)
df_sym = lambdify(x_s, df_sym,  "numpy")


In [ ]:

plt.plot(x, y, label="f")
plt.plot(x[1:-1], dydx[1:-1], lw=4, label="f' Num. Approx.")
plt.plot(x, df_sym(x), "--", label="f'")
plt.legend()
plt.show()

### Advanced numerical differenciation with [numdifftools](https://github.com/pbrod/numdifftools)

The numdifftools library addresses numerical differentiation with one or more variables:

- Derivative: Compute the derivatives of order 1 through 10 on any scalar function.
- directionaldiff: Compute directional derivative of a function of n variables
- Gradient: Compute the gradient vector of a scalar function of one or more variables.
- Jacobian: Compute the Jacobian matrix of a vector valued function of one or more variables.
- Hessian: Compute the Hessian matrix of all 2nd partial derivatives of a scalar function of one or more variables.
- Hessdiag: Compute only the diagonal elements of the Hessian matrix

To be completed

# Numerical Integration

- Principles [Patrick Walls course](https://patrickwalls.github.io/mathematicalpython/integration/).
- Library: [Scipy integrate](https://docs.scipy.org/doc/scipy/tutorial/integrate.html) package.


In [ ]:
import numpy as np

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
#import pystatsml.plot_utils

# Plot parameters
plt.style.use('seaborn-v0_8-whitegrid')
fig_w, fig_h = plt.rcParams.get('figure.figsize')
plt.rcParams['figure.figsize'] = (fig_w, fig_h * .5)
%matplotlib inline

## Methods based on sums of patches over intervals

Methods for integrating functions given fixed samples: $[(x_1, f(x_1)), ..., (x_i, f(x_i)), ...(x_N, f(x_N)]$.

[Riemann sums of rectangles](https://patrickwalls.github.io/mathematicalpython/integration/riemann-sums/) to approximate the area.
$$
\sum_{i=1}^N f(x_i^ * ) (x_i - x_{i-1}) \ \ , \ x_i^* \in [x_{i-1},x_i]
$$
The error is in $\mathcal{O}(\frac{1}{N})$

In [ ]:
f = lambda x : 1 / (1 + x ** 2)
a, b, N = 0, 5, 10
dx = (b - a) / N

x = np.linspace(a, b, N+1)
y = f(x)

x_ = np.linspace(a,b, 10*N+1) # 10 * N points to plot the function smoothly
plt.plot(x_, f(x_), 'b')
x_left = x[:-1] # Left endpoints
y_left = y[:-1]
plt.plot(x_left,y_left,'b.',markersize=10)
plt.bar(x_left,y_left,width=dx, alpha=0.2,align='edge',edgecolor='b')
plt.title('Left Riemann Sum, N = {}'.format(N))

Compute Riemann sums with 100 points

In [ ]:
a, b, N = 0, 5, 50
dx = (b - a) / N
x = np.linspace(a, b, N+1)

y = f(x)
print("Integral:", np.sum(f(x[:-1]) * np.diff(x)))

**[Trapezoid Rule](https://patrickwalls.github.io/mathematicalpython/integration/trapezoid-rule/)** sum the trapezoids connecting the points. The error is in $\mathcal{O}(\frac{1}{N^2})$. Use
[scipy.integrate.trapezoid](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.trapezoid.html) function:

In [ ]:
from scipy import integrate
integrate.trapezoid(f(x[:-1]), dx=dx)

**[Simpson's rule](https://patrickwalls.github.io/mathematicalpython/integration/simpsons-rule/)** uses a quadratic polynomial on each subinterval of a partition to approximate the function and to compute the definite integral. The error is in $\mathcal{O}(\frac{1}{N^4})$.
Use
[scipy.integrate.simpson](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.simpson.html#scipy.integrate.simpson) function:

In [ ]:
from scipy import integrate
integrate.simpson(f(x[:-1]), dx=dx)

**[Gauss-Legendre Quadrature](https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_quadrature)** approximate the integral of a function as a weighted sum of Legendre polynomials.

Methods for Integrating functions given function object $f()$ that could be evaluated for any value $x$ in a range $[a, b]$.

Use
[scipy.integrate.quad](https://docs.scipy.org/doc/scipy/tutorial/integrate.html#general-integration-quad) function. The first argument to `quad` is a “callable” Python object (i.e., a function, method, or class instance). Notice the use of a lambda- function in this case as the argument. The next two arguments are the limits of integration.

In [ ]:
import scipy.integrate as integrate
integrate.quad(f, a=a, b=b)

The return values are the estimated of the integral and the estimate of the absolute integration error. 